In [1]:
import jsonlines

In [2]:
all_file = './24-08-28/all.jsonl'
all_data = []
with open(all_file, 'r', encoding='utf-8') as file:
    for item in jsonlines.Reader(file):
        all_data.append(item)

In [32]:
def convert_formats(data):
    converted_data = []
    entity_id_to_name = {}
    entity_name_to_type = {}
    for doc in data:
        temp = {}
        temp['id'] = doc['id']
        temp['test'] = doc['text']
        temp_total = []
        temp_entity = []
        temp_realtion = []

        for e in doc['entities']:
            identifier_name = e['id']
            type_name = e['label']
            start_point = e['start_offset']
            end_point = e['end_offset']
            temp_entity.append({
                'id_name': identifier_name,
                'type_name': type_name,
                'start_offset': start_point,
                'end_offset': end_point
            })
        for re_test in doc['relations']:
            id_name = re_test['id']
            from_id = re_test['from_id']
            to_id = re_test['to_id']
            relation_type = re_test['type']
            temp_realtion.append({
                'id_name': id_name,
                'type_name': relation_type,
                'from_id': from_id,
                'to_id': to_id
            })
        temp['entity_list'] = temp_entity
        temp['relation_list'] = temp_realtion
        converted_data.append(temp)
    return {
        'data': converted_data
    }

def combine_text(data):
    test_list = []
    total_entity_list = []
    total_relation_list = []

    for m in data:
        temp_test = m['test']
        temp_entities = []
        temp_relations = []
        entity_list = m['entity_list']
        relation_list = m['relation_list']
        id_to_name = {}
        name_to_type = {}

        if entity_list == [] or entity_list == {} or entity_list == '' or entity_list == None:
            temp_entities.append({
                'id_name': '',
                'type_name': '',
                'test_name': ''
            })
        else:
            for n in entity_list:
                temp_entities.append({
                    'id_name': n['id_name'],
                    'type_name': n['type_name'],
                    'test_name': temp_test[n['start_offset']:n['end_offset']]
                })
                id_to_name[n['id_name']] = temp_test[n['start_offset']:n['end_offset']]
                name_to_type[temp_test[n['start_offset']:n['end_offset']]]= n['type_name']
        total_entity_list.append(temp_entities)
        test_list.append({
            'id': m['id'],
            'test': '\n'.join(temp_test.split('\n')[1::])
        })

        if relation_list == [] or relation_list == {} or relation_list == '' or relation_list == None:
            temp_relations.append({
                'id_name': '',
                'type_name': '',
                'entity1_name': '',
                'entity1_type': '',
                'entity2_name': '',
                'entity2_type': ''
            })
        else:
            for r in relation_list:
                temp_relations.append({
                    'id_name': r['id_name'],
                    'type_name': r['type_name'].replace('medication-information_frequency', 'medication information_frequency')\
                        .replace('Causes_disease', 'causes_disease').replace('symptom_oralpart', 'symptom_oral part')\
                            .replace('disease_oral-part', 'disease_oral part').replace('materials_use_as_medicine', 'materials_use as_medicine'),
                    'entity1_name': id_to_name[r['from_id']],
                    'entity1_type': name_to_type[id_to_name[r['from_id']]],
                    'entity2_name': id_to_name[r['to_id']],
                    'entity2_type': name_to_type[id_to_name[r['to_id']]]
                })
        total_relation_list.append(temp_relations)
    return {
        'test_list': test_list, 
        'total_entity_list': total_entity_list, 
        'total_relation_list': total_relation_list
    }

def count_num(test_list, total_entity_list, total_relation_list):
    count_relation = {}
    count_entity = {}
    for i in range(len(test_list)):
        for e in total_entity_list[i]:
            if e['type_name'] not in count_entity.keys():
                count_entity[e['type_name']] = 1
            else:
                count_entity[e['type_name']] += 1
        for r in total_relation_list[i]:
            if r['type_name'] not in count_relation.keys():
                count_relation[r['type_name']] = 1
            else:
                count_relation[r['type_name']] += 1
    return count_relation, count_entity

def check_text(test_list, total_entity_list, total_relation_list):
    e_source = 'none'
    r_source = 'prevent'
    output_text = []
    for i in range(len(test_list)):
        for e in total_entity_list[i]:
            if e['type_name'] == e_source:
                output_text.append(str(test_list[i]['test']) + '\nentities: ' + str(total_entity_list[i]) + '\nrelations: ' + str(total_relation_list[i]))
        for r in total_relation_list[i]:
            if r['type_name'] == r_source:
                output_text.append(str(test_list[i]['test']) + '\nentities: ' + str(total_entity_list[i]) + '\nrelations: ' + str(total_relation_list[i]))
    return output_text




In [33]:
count_relation, count_entity = count_num(**combine_text(**convert_formats(all_data)))
with open('./count.txt', 'w') as f:
    f.write(str(count_relation) + '\n\n' + str(count_entity))

In [34]:
output = check_text(**combine_text(**convert_formats(all_data)))
print(output)
with open('./check.txt', 'w') as f:
    f.write('\n-------------------------------------------------------------------------------------\n'.join(output))

["\nA second mechanism of protection from caries is the incorporation of fluoride into bacterial biofilms where it inhibits enolase. Enolase catalyzes the production of phosphoenolpyruvate, the precursor of lactate in glycolysis, from 2-phosphoglycerate during glycolysis (7 – see also  In addition, oral bacterial uptake of mono- and disaccharides mostly utilizes the phosphoenolpyruvate transport system to transfer them into the cytosol (Sect. 15.2.2). Fluoride therefore inhibits not only lactic acid production, but also the phosphoenolpyruvate transport system-mediated uptake of saccharide substrates. In short, fluoride inhibits saccharolytic fermentation by many oral bacteria. Although high levels of fluoride (<3,000 ppm) kill most bacteria, there is little evidence that common levels of fluoride (1–10 ppm) alter the types of bacterial species or their relative concentrations in biofilms. Mutans and other streptococci in the biofilm may switch to asaccharolytic fermentation (Sect. 1.3